# Showcase of Jupyter and EnOSlib integration (♥JupEnOS♥)

Based on the example [Grid'5000 subnet reservation](https://discovery.gitlabpages.inria.fr/enoslib/tutorials/grid5000.html#subnet-reservation)




In [1]:
import logging
from pathlib import Path

import enoslib as en

logging.basicConfig(level=logging.INFO)

job_name = "♥JupEnOS♥"


prod_network = en.G5kNetworkConf(id="n1", type="prod", roles=["my_network"], site="rennes")
conf = (
    en.G5kConf.from_settings(job_name=job_name, job_type="allow_classic_ssh")
    .add_network_conf(prod_network)
    .add_network(
        id="not_linked_to_any_machine",
        type="slash_16",
        roles=["my_subnet"],
        site="rennes",
    )
    .add_machine(
        roles=["control"], cluster="paranoia", nodes=1, primary_network=prod_network
    )
    .finalize()
)

A quick look to the Configuration object

In [2]:
conf

Conf@0x7f4a483ca410
{
    "dhcp": true,
    "force_deploy": false,
    "env_name": "debian10-x64-nfs",
    "job_name": "\u2665JupEnOS\u2665",
    "job_type": "allow_classic_ssh",
    "key": "/home/msimonin/.ssh/id_rsa.pub",
    "queue": "default",
    "walltime": "02:00:00",
    "resources": {
        "machines": [
            {
                "roles": [
                    "control"
                ],
                "primary_network": "n1",
                "secondary_networks": [],
                "cluster": "paranoia",
                "nodes": 1
            }
        ],
        "networks": [
            {
                "id": "n1",
                "type": "prod",
                "roles": [
                    "my_network"
                ],
                "site": "rennes"
            },
            {
                "id": "not_linked_to_any_machine",
                "type": "slash_16",
                "roles": [
                    "my_subnet"
                ],
                "site": "rennes"
            }
        ]
    }
}

## Let's get the actual resources

In [3]:
provider = en.G5k(conf)
roles, networks = provider.init()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from grenoble
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Submitting {'name': '♥JupEnOS♥', 'types': ['allow_classic_ssh'], 'resources': "{cluster='paranoia'}/nodes=1+slash_16=1,walltime=02:00:00", 'command': 'sleep 31536000', 'queue': 'default'} on rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1714656 on rennes [2021-07-03 16:09:02]
INFO:enoslib.infra.enos_g5k.g5k_api_utils:All jobs are Running !


## Exploring your actual roles

Looking at the roles, you can get some initial info about how the actual machines have been grouped together.

Exploring the nested structure allows you to get some of the initial information about the resources.

In [4]:
roles

{'control': [Host(address='paranoia-7.rennes.grid5000.fr', alias='paranoia-7.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())]}

## This can be done for networks too !

Networks are grouped by role names.


In G5k specific case an IPv4 network may have an IPv6 companion. This is the case for the production network and both will be returned as usable for your experiment.

You can also observed that a /16 on G5k is made of plenty /22 networks from which a pool of IPs/Macs is availabel for your experiment.

In [5]:
networks

{'my_network': [<enoslib.infra.enos_g5k.objects.G5kEnosProd4Network object at 0x7f4a2210efd0>, <enoslib.infra.enos_g5k.objects.G5kEnosProd6Network object at 0x7f4a2210ed10>], 'my_subnet': [<enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2210ee50>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2211fa90>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22121dd0>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22121f10>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22121e50>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22121590>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22121b50>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22121190>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22121f90>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22121d10>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22ac7210>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22ac7190>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22ac7a50>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22ac7910>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22125a10>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22125f50>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22125190>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22125310>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22125610>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22125b10>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22125910>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22125450>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22125810>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22125990>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a22125dd0>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212b110>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212b250>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212b390>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212b4d0>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212b610>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212b750>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212b890>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212b9d0>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212bb10>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212bc50>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212bd90>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212bed0>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212d050>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212d190>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212d2d0>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212d410>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212d550>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212d690>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212d7d0>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212d910>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212da50>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f4a2212db90>, <enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwor

## Roles can be synced to get more information

The rationale behind syncing the node information includes:
- getting contextualized network information for each node
- getting hardware information (e.g. cpu counts)
- (get in touch if you need extra information in here :) )


In [6]:

roles = en.sync_info(roles, networks)

[WARNING]: No inventory was parsed, only implicit localhost is available



PLAY [all] *********************************************************************

TASK [hostname] ****************************************************************
changed: [paranoia-7.rennes.grid5000.fr]

PLAY RECAP *********************************************************************
paranoia-7.rennes.grid5000.fr : ok=1    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   


PLAY [Gather facts for all hosts] **********************************************

TASK [Gathering Facts] *********************************************************


[DEPRECATION WARNING]: Distribution debian 10.9 on host 
paranoia-7.rennes.grid5000.fr should use /usr/bin/python3, but is using 
/usr/bin/python for backward compatibility with prior Ansible releases. A 
future Ansible release will default to using the discovered platform python for
 this host. See https://docs.ansible.com/ansible/2.10/reference_appendices/inte
rpreter_discovery.html for more information. This feature will be removed in 
version 2.12. Deprecation warnings can be disabled by setting 
deprecation_warnings=False in ansible.cfg.


ok: [paranoia-7.rennes.grid5000.fr]

TASK [setup] *******************************************************************
ok: [paranoia-7.rennes.grid5000.fr]

PLAY [Utils functions] *********************************************************

TASK [Gathering Facts] *********************************************************
ok: [paranoia-7.rennes.grid5000.fr]

TASK [utils : include] *********************************************************
included: /home/msimonin/workspace/repos/enoslib/enoslib/ansible/roles/utils/tasks/check_network.yml for paranoia-7.rennes.grid5000.fr

TASK [utils : Dump network information in a file] ******************************
changed: [paranoia-7.rennes.grid5000.fr]

TASK [utils : Create the fake interfaces] **************************************
skipping: [paranoia-7.rennes.grid5000.fr]

PLAY RECAP *********************************************************************
paranoia-7.rennes.grid5000.fr : ok=5    changed=1    unreachable=0    failed=0    skipped=1    rescu

In [7]:
roles

ip
fe80::a236:9fff:fe28:a914/64
172.16.100.7/20
ip
::1/128
127.0.0.1/8


### Of course don't forget to release your machines

In [8]:
provider.destroy()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from grenoble
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading 1714656 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading ♥JupEnOS♥ from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Killing the job (rennes, 1714656)
